# Implémentation de quelques algorithmes de trading
1.Importation des bibliothèques
pandas : la manipulation des Datafrmes
matplotlib : production des visuels
yfinance : vise à résoudre ce problème en offrant un moyen fiable,avec la possibilité de télécharger des données de marché historiques à partir de Yahoo!finance.
TA-Lib :est largement utilisée par les développeurs de logiciels de trading nécessitant d'effectuer une analyse technique des données des marchés financiers.


Comprend plus de 150 indicateurs tels que ADX, MACD, RSI, stochastique, bandes de Bollinger, etc.

 

Reconnaissance de modèle de chandelier

  API open source pour C/C++, Java, Perl, Python et .NET 100 % géré

**REMARQUE :** L'installation de cette bibliothèque est un peu laborieuse il est préconisé de suivre les étapes préscrites dans ce tuto : https://blog.quantinsti.com/install-ta-lib-python/

In [1]:
pip install yfinance

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install backtrader

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install talib

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement talib (from versions: none)
ERROR: No matching distribution found for talib


In [4]:
import talib
import yfinance
from matplotlib import pyplot as plt
import datetime
import backtrader as bt

# L'indicateur RSI

In [5]:
import pandas as pd

In [9]:
data_finance=pd.read_csv('CAC40.csv')

In [10]:
data_finance

,Date,Open,High,Low,Close,Adj Close,Volume
0,2021-10-11,6546.069824,6574.220215,6522.549805,6570.540039,6570.540039,68968500
1,2021-10-12,6495.149902,6548.109863,6491.580078,6548.109863,6548.109863,69996000
2,2021-10-13,6539.609863,6606.359863,6510.270020,6597.379883,6597.379883,75938000
3,2021-10-14,6651.180176,6692.939941,6618.790039,6685.209961,6685.209961,71368900
4,2021-10-15,6714.259766,6746.810059,6703.259766,6727.520020,6727.520020,84105700
...,...,...,...,...,...,...,...
254,2022-10-04,5859.450195,6039.689941,5852.919922,6039.689941,6039.689941,95391900
255,2022-10-05,6006.120117,6034.930176,5953.549805,5985.459961,5985.459961,75121600
256,2022-10-06,6004.109863,6018.470215,5917.270020,5936.419922,5936.419922,63735200
257,2022-10-07,5911.959961,5956.950195,5855.660156,5866.939941,5866.939941,65700300


In [11]:
data_finance.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 259 entries, 0 to 258
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       259 non-null    object 
 1   Open       259 non-null    float64
 2   High       259 non-null    float64
 3   Low        259 non-null    float64
 4   Close      259 non-null    float64
 5   Adj Close  259 non-null    float64
 6   Volume     259 non-null    int64  
dtypes: float64(5), int64(1), object(1)
memory usage: 14.3+ KB


In [12]:
data_finance.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2021-10-11,6546.069824,6574.220215,6522.549805,6570.540039,6570.540039,68968500
1,2021-10-12,6495.149902,6548.109863,6491.580078,6548.109863,6548.109863,69996000
2,2021-10-13,6539.609863,6606.359863,6510.270020,6597.379883,6597.379883,75938000
3,2021-10-14,6651.180176,6692.939941,6618.790039,6685.209961,6685.209961,71368900
4,2021-10-15,6714.259766,6746.810059,6703.259766,6727.520020,6727.520020,84105700


In [13]:
data_finance["Open"].max()

7320.47998

In [14]:
data_finance["Low"].min()

5628.419922

In [15]:
data_finance["High"].max()

7384.859863

In [16]:
data_finance["Close"].median()

6537.410156

In [17]:
data_finance.std()

C:\Users\hp\AppData\Local\Temp/ipykernel_1924/2168718258.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  data_finance.std()


Open         4.134917e+02
High         4.041315e+02
Low          4.180945e+02
Close        4.121345e+02
Adj Close    4.121345e+02
Volume       3.477940e+07
dtype: float64

In [ ]:
from datetime import datetime
import backtrader as bt
from chapter_2_utils import MyBuySell

# create a Stratey
class RsiSignalStrategy(bt.SignalStrategy):
    params = dict(rsi_periods=14, rsi_upper=70, rsi_lower=30, rsi_mid=50)

    def __init__(self):

        # add RSI indicator
        rsi = bt.indicators.RSI(period=self.p.rsi_periods,
                                upperband=self.p.rsi_upper,
                                lowerband=self.p.rsi_lower)

        # add RSI from TA-lib just for reference
        bt.talib.RSI(self.data, plotname='TA_RSI')

        # long condition (with exit)
        rsi_signal_long = bt.ind.CrossUp(rsi, self.p.rsi_lower, plot=False)
        self.signal_add(bt.SIGNAL_LONG, rsi_signal_long)
        self.signal_add(bt.SIGNAL_LONGEXIT, -(rsi > self.p.rsi_mid))

        # short condition (with exit)
        rsi_signal_short = -bt.ind.CrossDown(rsi, self.p.rsi_upper, plot=False)
        self.signal_add(bt.SIGNAL_SHORT, rsi_signal_short)
        self.signal_add(bt.SIGNAL_SHORTEXIT, rsi < self.p.rsi_mid)

# create a Data Feed
data = bt.feeds.YahooFinanceData(dataname='FB',
                                 fromdate=datetime(2018, 1, 1),
                                 todate=datetime(2018, 12, 31))

# create a Cerebro entity
cerebro = bt.Cerebro(stdstats = False)

# # set up the backtest
cerebro.addstrategy(RsiSignalStrategy)
cerebro.adddata(data)
cerebro.broker.setcash(1000.0)
cerebro.broker.setcommission(commission=0.001)
cerebro.addobserver(MyBuySell)
cerebro.addobserver(bt.observers.Value)
cerebro.run()

cerebro.plot(iplot=True, volume=False)